## Re-examine SF_Uncert metric in DDF regarding season length (LogT batch)

In [1]:
# development code
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import glob
import os, sys

In [3]:
# add the path the scripts
sys.path.insert(0, "../scripts/")

In [4]:
# import rubin_sim python modules
import rubin_sim.maf.db as db
import rubin_sim.maf.metrics as metrics
import rubin_sim.maf.slicers as slicers
import rubin_sim.maf.stackers as stackers
import rubin_sim.maf.plots as plots
import rubin_sim.maf.metricBundles as metricBundles
import rubin_sim.maf as maf
from rubin_sim.maf.utils import m52snr

# footprint utils
from rubin_sim.scheduler.utils import footprints

# import convenience functions
from opsimUtils import *

# import custom stacker/metrics from script
from agnstructure import *

# print version
import rubin_sim

rubin_sim.__version__

'0.10.1.dev64+gc32f68b'

### 1. Pull rolling opsims to evaluate 

In [5]:
families = maf.archive.get_family_descriptions()
family_list = families.index.values

# pull roll runs
fam = ["ddf season length"]
baseline_run = families.loc["ddf season length", "reference"]
ddf_runs = np.concatenate([[baseline_run], families.explode("run").loc[fam, "run"]])
ddf_runs

array(['baseline_v2.1_10yrs', 'ddf_season_length_slf0.10_v2.1_10yrs',
       'ddf_season_length_slf0.15_v2.1_10yrs',
       'ddf_season_length_slf0.20_v2.1_10yrs',
       'ddf_season_length_slf0.25_v2.1_10yrs',
       'ddf_season_length_slf0.30_v2.1_10yrs',
       'ddf_season_length_slf0.35_v2.1_10yrs'], dtype=object)

### 2. Test SF Uncert on the baseline

In [6]:
dbDir = rubin_sim.data.get_data_dir()
outDir = "/home/jovyan/mount/ResearchData/MAFOutput/ddf_tgap/ResultsDb"
metricDataPath = "/home/jovyan/mount/ResearchData/MAFOutput/ddf_tgap/MetricData"

if not os.path.exists(os.path.abspath(outDir)):
    os.makedirs(os.path.abspath(outDir))

if not os.path.exists(os.path.abspath(metricDataPath)):
    os.makedirs(os.path.abspath(metricDataPath))

In [7]:
# create dict for storing dbs
opSimDbs, resultDbs = connect_dbs(dbDir, outDir, dbRuns=ddf_runs)

In [8]:
ddfFields = ['COSMOS', 'XMM-LSS', 'ELAISS1', 'ECDFS', 'EDFS']
agn_m5 = {"u": 22.89, "g": 23.94, "r": 23.5, "i": 22.93, "z": 22.28, "y": 21.5}

#### 2.1 Default 10 bins

In [9]:
# metric name template
metricNameTemp1 = 'logTGap_{}_{}_10'

# custom sf bins
nbin = 10
mybins = np.logspace(0, np.log10(3650), nbin+1)
myweights = np.full(nbin, 1/nbin)

In [10]:
for run in ddf_runs:
    print(f'Runing metrics on Opsim: {run}!')
    print('*****************************************')
    bdict1 = {}

    for ddf in ddfFields:
        # custom slicer
        nside = 128
        radius = 1.8
        hp_mask = ddf_hp_mask(ddf, radius, nside)
        slicer = slicers.HealpixSubsetSlicer(nside, np.where(hp_mask == 1)[0])
        slicer.slicerName = 'HealpixSlicer'  # hack maf
    
        for band in 'ugrizy':
            metricName = metricNameTemp1.format(ddf, band)
            sfuncertdev = SFUncertMetricDev(mag=agn_m5[band], 
                                            bins=mybins, weight=myweights, 
                                            snr_cut=0, metricName=metricName)
            constraint = f'filter = "{band}"'

            # create bundle
            bdict1[metricName] = metricBundles.MetricBundle(sfuncertdev, slicer, 
                                                            constraint, 
                                                            runName = run)

    gp = metricBundles.MetricBundleGroup(bdict1, opSimDbs[run], outDir=metricDataPath,
                                         resultsDb=resultDbs[run], verbose=False)
    gp.runAll()

Runing metrics on Opsim: baseline_v2.1_10yrs!
*****************************************
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes


KeyboardInterrupt: 

#### Read Results from Disk & Make Plots

In [ ]:
# retrieve metricBundles for each opsim run and store them in a dictionary
bundleDicts = {}

for runName in resultDbs:
    bundleDicts[runName] = bundleDictFromDisk(resultDbs[runName], runName, \
                                              metricDataPath)

In [ ]:
# plot COSMOS in g band
for run in bundleDicts:
    bDict = bundleDicts[run]
    key = [metricKey for metricKey in bDict.keys() 
           if metricKey[1] == 'logTGap_XMM-LSS_g_10'][0]
    metric = bDict[key]
    mask = metric.metricValues.mask
    data = metric.metricValues.data[~mask]
    nobs_md = np.median(np.vstack(data), axis=0)
    
    plt.stairs(np.log10(nobs_md), np.log10(mybins), label=run)
    
plt.xlabel('Log($\Delta t$)')
plt.yscale('log')
plt.legend(loc=3, bbox_to_anchor=(1.,0.))
plt.ylabel('N pairs')
plt.title('XMM-LSS_g; 10 bins')

#### 2.2 Default 20 bins

In [ ]:
# metric name template
metricNameTemp2 = 'logTGap_{}_{}_20'

# custom sf bins
nbin = 20
mybins = np.logspace(0, np.log10(3650), nbin+1)
myweights = np.full(nbin, 1/nbin)

In [ ]:
for run in ddf_runs:
    print(f'Runing metrics on Opsim: {run}!')
    print('*****************************************')
    bdict2 = {}
    
    for ddf in ddfFields:
        # custom slicer
        nside = 128
        radius = 1.8
        hp_mask = ddf_hp_mask(ddfFields[0], radius, nside)
        slicer = slicers.HealpixSubsetSlicer(nside, np.where(hp_mask == 1)[0])
        slicer.slicerName = 'HealpixSlicer'  # hack maf

        for band in 'ugrizy':
            metricName = metricNameTemp2.format(ddf, band)
            sfuncertdev = SFUncertMetricDev(mag=agn_m5[band], 
                                            bins=mybins, weight=myweights, 
                                            snr_cut=0, metricName=metricName)
            constraint = f'filter = "{band}"'

            # create bundle
            bdict2[metricName] = metricBundles.MetricBundle(sfuncertdev, slicer, 
                                                            constraint, 
                                                            runName = run)

    gp = metricBundles.MetricBundleGroup(bdict2, opSimDbs[run], outDir=metricDataPath, 
                                         resultsDb=resultDbs[run], verbose=False)
    gp.runAll()

In [ ]:
# plot COSMOS in g band
for run in bundleDicts:
    bDict = bundleDicts[run]
    key = [metricKey for metricKey in bDict.keys() 
           if metricKey[1] == 'logTGap_XMM-LSS_g_20'][0]
    metric = bDict[key]
    mask = metric.metricValues.mask
    data = metric.metricValues.data[~mask]
    nobs_md = np.median(np.vstack(data), axis=0)
    
    plt.stairs(np.log10(nobs_md), np.log10(mybins), label=run)
    
plt.xlabel('Log($\Delta t$)')
plt.yscale('log')
plt.legend(loc=3, bbox_to_anchor=(1.,0.))
plt.ylabel('N pairs')
plt.title('XMM-LSS_g; 20 bins')